In [1]:
import pronouncing
from itertools import product
import functools
import operator
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from tqdm import tqdm_notebook as tqdm
import string
import gensim

In [4]:
model = gensim.models.KeyedVectors.load_word2vec_format('InferSent/dataset/fastText/crawl-300d-2M.vec')  

In [63]:
stop = set(stopwords.words('english'))

def synonyms(word):
    if not INCLUDE_STOP and word in stop:
        return [word]
    
    most_similar =model.most_similar(positive=[word], topn=50)
    
    return list(set([word[0].lower() for word in most_similar if len(pronouncing.phones_for_word(word[0])) > 0]))[:50]

synonyms('slow')

/home/matthew/.local/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


['slowing',
 'rapid',
 'slowest',
 'slow',
 'slowed',
 'plodding',
 'slowness',
 'fast',
 'slows',
 'slower',
 'steady',
 'sluggish']

In [10]:
def meter(phrase):
    phrase = phrase.translate(str.maketrans({a:None for a in string.punctuation})) # strip punctuation
    syllables = 0 
    stresses = []
    for word in word_tokenize(phrase):
#         try:
        phonemes = pronouncing.phones_for_word(word)[0]
        syllables += pronouncing.syllable_count(phonemes)
        stresses.append(list(map(int, pronouncing.stresses(phonemes))))
#         except IndexError:
# #             print(f'\tError: word without phoneme found: "{word}"')
#             syllables += 1
#             stresses.append([1])

    return (syllables, stresses)

In [11]:
meter('that was amazing')

(5, [[1], [1], [0, 1, 0]])

In [12]:
def check_iambic(line):
    syllables, stresses = meter(line)
    
    if syllables is not 10:
        return False
    
    expected_next = False # not stressed
    
    for stress in stresses:
        if len(stress) == 1:
            expected_next = not expected_next
            continue # one syllable can be stressed or not stressed
        
        for syllable in stress:
            if syllable == 2:
                expected_next = not expected_next
                continue # lightly stressed, can be either
                
            if (syllable == 1) == expected_next:
                expected_next = not expected_next
                continue
            else:
                return False
    return True

In [13]:
print(check_iambic('How do I love thee? Let me count the ways.'))
print(check_iambic('Is this smarter than simply syllables?'))

True
False


In [14]:
test_poem = \
'''How do I love thee? Let me count the ways.
I love thee to the depth and breadth and height
My soul can reach, when feeling out of sight
For the ends of being and ideal grace.
I love thee to the level of every day's
Most quiet need, by sun and candle-light.
I love thee freely, as men strive for right.
I love thee purely, as they turn from praise.
I love thee with the passion put to use
In my old griefs, and with my childhood's faith.
I love thee with a love I seemed to lose
With my lost saints. I love thee with the breath,
Smiles, tears, of all my life; and, if God choose,
I shall but love thee better after death.
'''.split('\n')
for line in test_poem:
    try:
        if not check_iambic(line):
            print(line)
    except:
        pass

For the ends of being and ideal grace.
I love thee to the level of every day's



In [58]:
def make_iambic(line, return_early=10):
    words = line.split(' ') # word_tokenize(line)
    word_synonyms = list(map(synonyms, words))
    
    for i, syns in enumerate(word_synonyms):
        if len(syns) == 0:
            word_synonyms[i] = [words[i]]
            
    iambic = []
    # https://stackoverflow.com/questions/32074543/how-to-get-the-length-of-an-itertools-product
    n_possible = functools.reduce(operator.mul, map(len, word_synonyms), 1)
    for possible in tqdm(product(*word_synonyms), total=n_possible):
        potential = ' '.join(possible)
        if check_iambic(potential):
            iambic.append(potential)
            
            if return_early and len(iambic) >= return_early:
                return iambic
    return iambic

In [64]:
INCLUDE_STOP = False
make_iambic('For the ends of being and ideal grace.')

/home/matthew/.local/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


HBox(children=(IntProgress(value=0, max=2496), HTML(value='')))

['in the begins of being and best grace',
 'in the begins of being and good grace',
 'in the begin of being and best grace',
 'in the begin of being and good grace',
 'in the concludes of being and best grace',
 'in the concludes of being and good grace',
 'with the begins of being and best grace',
 'with the begins of being and good grace',
 'with the begin of being and best grace',
 'with the begin of being and good grace']

In [ ]:
corpus = word_tokenize(open('sermon_mount_NIV.txt').read())

In [ ]:
def find_iambic(corpus):
    